# 初识 Zigent：构建你的又一个智能搜索代理

## 基础参数配置

In [ ]:
import os
from dotenv import load_dotenv
# 加载环境变量
load_dotenv() 

print(os.getenv('ZHIPU_API_KEY'))
# 从环境变量中读取api_key
api_key = os.getenv('ZHIPU_API_KEY')
chat_model = "glm-4-flash"
emb_model = "embedding-2"
base_url = "https://open.bigmodel.cn/api/paas/v4"



9d9338dcf6dcc6d703614ba53fce2e26.AdQUvVowRBvlh1cR


## 引入 zigent 相关的方法如Action、Agent、大模型配置等以及 duckduckgo_search ：

In [5]:
from typing import List
from zigent.agents import ABCAgent, BaseAgent
from zigent.llm.agent_llms import LLM
from zigent.commons import TaskPackage
from zigent.actions.BaseAction import BaseAction
# from zigent.logging.multi_agent_log import AgentLogger
from duckduckgo_search import DDGS

In [8]:
llm = LLM(api_key=api_key, base_url=base_url, model_name=chat_model)
response = llm.run("你是谁？")
print(response)

我是一个名为 ChatGLM 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。


In [9]:
class DuckSearchAction(BaseAction):
    def __init__(self) -> None:
        action_name = "DuckDuckGo_Search"
        action_desc = "Using this action to search online content."
        params_doc = {"query": "the search string. be simple."}
        self.ddgs = DDGS()
        super().__init__(
            action_name=action_name, 
            action_desc=action_desc, 
            params_doc=params_doc,
        )

    def __call__(self, query):
        results = self.ddgs.chat(query)
        return results

In [10]:
search_action = DuckSearchAction()
results = search_action("什么是 agent")
print(results)

“Agent”这个词在不同的领域有不同的含义。以下是一些常见的解释：

1. **一般意义**：在日常用语中，agent指的是一个代理人或代表，负责代表他人进行某种活动或决策。

2. **计算机科学**：在人工智能和计算机科学中，agent通常指的是一种能够自主执行任务的程序或系统。例如，智能代理（intelligent agent）可以根据环境的变化做出决策和行动。

3. **商业**：在商业领域，agent可以指代中介或代理商，他们代表公司或个人进行交易或谈判。

4. **生物学**：在生物学中，agent可以指代某种能够引起生物反应的物质，如病原体或药物。

5. **法律**：在法律上，agent是指被授权代表他人行事的人，通常涉及合同或法律事务。

具体的含义取决于上下文。如果你有特定的领域或背景，请告诉我，我可以提供更详细的信息。


In [11]:
class DuckSearchAgent(BaseAgent):
    def __init__(
        self,
        llm: LLM,
        actions: List[BaseAction] = [DuckSearchAction()],
        manager: ABCAgent = None,
        **kwargs
    ):
        name = "duck_search_agent"
        role = "You can answer questions by using duck duck go search content."
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager
        )

In [12]:
def do_search_agent():
    # 创建代理实例
    search_agent = DuckSearchAgent(llm=llm)

    # 创建任务
    task = "what is the found date of microsoft"
    task_pack = TaskPackage(instruction=task)

    # 执行任务并获取响应
    response = search_agent(task_pack)
    print("response:", response)

if __name__ == "__main__":
    do_search_agent()

Agent duck_search_agent receives the following TaskPackage:
[
	Task ID: 6ea83389-ff97-42d3-87b1-817aa4e3b835
	Instruction: what is the found date of microsoft
]
====duck_search_agent starts execution on TaskPackage 6ea83389-ff97-42d3-87b1-817aa4e3b835====
Agent duck_search_agent takes 0-step Action:
{
	name: DuckDuckGo_Search[{"query": "found date of Microsoft"}}
	params: {}
}
Observation: "This is the wrong action to call. Please check your available action list.
Agent duck_search_agent takes 1-step Action:
{
	name: Think
	params: {'response': "The correct action to use in this scenario is 'DuckDuckGo_Search' with the appropriate query to find the founding date of Microsoft."}
}
Observation: OK
Agent duck_search_agent takes 2-step Action:
{
	name: DuckDuckGo_Search
	params: {'query': 'founding date of Microsoft'}
}
Observation: Microsoft was founded on April 4, 1975.
Agent duck_search_agent takes 3-step Action:
{
	name: Finish
	params: {'response': 'Microsoft was founded on April 4, 1

# 使用智谱AI的web_search

In [13]:
import os
import json
import requests
from dotenv import load_dotenv
from typing import List
from zigent.agents import BaseAgent, ABCAgent
from zigent.llm.agent_llms import LLM
from zigent.commons import TaskPackage
from zigent.actions.BaseAction import BaseAction
from zhipuai import ZhipuAI
from datetime import datetime

# 加载环境变量
load_dotenv()
ZHIPU_API_KEY = os.getenv('ZHIPU_API_KEY')
ZHIPU_BASE_URL = os.getenv('ZHIPU_BASE_URL')
model_name = "glm-4-flash"
llm = LLM(api_key=ZHIPU_API_KEY, base_url=ZHIPU_BASE_URL, model_name=model_name)
llm.run("你谁")

'我是一个名为 ChatGLM 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。'

In [6]:
# 定义Zhipu Web Search工具
def zhipu_web_search_tool(query: str) -> str:
    """
    使用智谱AI的GLM-4模型进行联网搜索，返回搜索结果的字符串。
    
    参数:
    - query: 搜索关键词

    返回:
    - 搜索结果的字符串形式
    """
    # 初始化客户端
    client = ZhipuAI(api_key=ZHIPU_API_KEY)

    # 获取当前日期
    current_date = datetime.now().strftime("%Y-%m-%d")

    print("current_date:", current_date)
    
    # 设置工具
    tools = [{
        "type": "web_search",
        "web_search": {
            "enable": True
        }
    }]

    # 系统提示模板，包含时间信息
    system_prompt = f"""你是一个具备网络访问能力的智能助手，在适当情况下，优先使用网络信息（参考信息）来回答，
    以确保用户得到最新、准确的帮助。当前日期是 {current_date}。"""
        
    # 构建消息
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]
        
    # 调用API
    response = client.chat.completions.create(
        model="glm-4-flash",
        messages=messages,
        tools=tools
    )
    
    # 返回结果
    return response.choices[0].message.content

In [7]:
class ZhipuSearchAction(BaseAction):
    def __init__(self) -> None:
        action_name = "Zhipu_Search"
        action_desc = "Using this action to search online content."
        params_doc = {"query": "the search string. be simple."}
        super().__init__(
            action_name=action_name,
            action_desc=action_desc,
            params_doc=params_doc,
        )

    def __call__(self, query):
        results = zhipu_web_search_tool(query)
        return results

In [8]:
class ZhipuSearchAgent(BaseAgent):
    def __init__(
        self,
        llm: LLM,
        actions: List[BaseAction] = [ZhipuSearchAction()],
        manager: ABCAgent = None,
        **kwargs
    ):
        name = "zhiu_search_agent"
        role = "You can answer questions by using Zhipu search content."
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager,
        )


In [14]:
def do_search_agent():
    # 创建代理实例
    search_agent = ZhipuSearchAgent(llm=llm)

    # 创建任务
    task = "2025年洛杉矶大火"
    task_pack = TaskPackage(instruction=task)

    # 执行任务并获取响应
    response = search_agent(task_pack)
    print(response)

if __name__ == "__main__":
    do_search_agent()

Agent zhiu_search_agent receives the following TaskPackage:
[
	Task ID: fbb8bdf8-aef6-4d45-924b-8a8278d29f0b
	Instruction: 2025年洛杉矶大火
]
====zhiu_search_agent starts execution on TaskPackage fbb8bdf8-aef6-4d45-924b-8a8278d29f0b====
Agent zhiu_search_agent takes 0-step Action:
{
	name: Zhipu_Search
	params: {'query': '2025年洛杉矶大火'}
}
current_date: 2025-02-16
Observation: 2025年洛杉矶大火是美国历史上最严重的自然灾害之一。这场大火发生在2025年1月初，起始于洛杉矶县太平洋帕利塞德地区的山区，随后迅速蔓延至多个区域，包括帕萨迪纳、圣费尔南多山谷以及沿海的马里布。火灾的原因主要是由于极端干燥的天气和强劲的圣安娜风，这两种因素共同导致了火势的迅速蔓延。

火灾造成了巨大的损失，包括超过12,000公顷的土地被烧毁，至少12,000栋建筑被摧毁或无法居住。据报道，火灾导致了27人死亡，近10万居民流离失所。这场大火的总经济损失估计可能高达500亿美元，是2008年全美许多州房地产泡沫导致全球经济崩溃事件的数倍。

这场火灾凸显了气候变化带来的严峻挑战，包括极端天气事件的频繁发生。随着气候变化的加剧，类似的大火可能会变得更加普遍和破坏性。此外，火灾也暴露了美国保险业在面对如此大规模灾害时的脆弱性，许多保险公司正在撤出在风险较高的地区的业务，导致保险供应紧张和保费上涨。

在火灾发生之后，加利福尼亚州及其他受灾地区的居民面临了漫长的重建之路，同时政府如何支付巨额的赔付成为一个关键问题。此外，这场大火也引起了全球金融体系的关注，因为如果高风险地区的气候保险泡沫破裂，可能会引发类似于2008年金融危机的连锁反应。

为了应对这样的自然灾害，专家呼吁采取更多的预防措施，包括改善植被管理、提高建筑物的防火标准，以及增强应急响应能力。
Agent zhiu_search_agent takes 1-step Action: